In [42]:
##### read input data #####
import pandas as pd 
import os

schedules = {}
for filename in os.listdir('./testdata'):

    # there has been the sign \xa0 in the test data
    with open('./testdata/'+filename, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    lines = [line.replace('\xa0', '') for line in lines]
    with open('./testdata/'+filename, 'w', encoding='utf-8') as file:
        file.writelines(lines)

    team = filename[:-4]
    raw_df = pd.read_csv('./testdata/'+filename,sep=";").dropna().drop('Schiedsrichter', axis=1)
    schedules[team] = raw_df

print(schedules['damen1'])

      Nr.   Tag             Datum                       Heim  \
0     5.0   1.0  21.09.2024 17:00         WINGS Leverkusen 2   
2     7.0   2.0  28.09.2024 18:00            DJK Frankenberg   
4    15.0   3.0  05.10.2024 18:00         SG Bergische Löwen   
6    19.0   4.0  02.11.2024 20:00            DJK Frankenberg   
8    30.0   5.0  09.11.2024 16:00    VfL AstroStars Bochum 2   
10   31.0   6.0  15.11.2024 20:30            DJK Frankenberg   
12   40.0   7.0  30.11.2024 18:30    New Basket Oberhausen 2   
14   43.0   8.0  07.12.2024 20:00            DJK Frankenberg   
16   50.0   9.0  14.12.2024 16:00                  TSV Hagen   
18   55.0  10.0  21.12.2024 16:00  Citybasket Recklinghausen   
20   61.0  11.0  18.01.2025 18:00            DJK Frankenberg   
22   71.0  12.0  25.01.2025 18:00            DJK Frankenberg   
24   73.0  13.0  01.02.2025 16:30                UBC Münster   
26   81.0  14.0  08.02.2025 18:00            DJK Frankenberg   
28   85.0  15.0  15.02.2025 18:00       

In [43]:
##### filter home schedules from schedules #####
home_schedules = {}
for team, schedule in schedules.items():
    home_schedules[team] = schedule[schedule['Heim'].str.contains('DJK Frankenberg')]

print(home_schedules['damen1'])

      Nr.   Tag             Datum             Heim                       Gast  \
2     7.0   2.0  28.09.2024 18:00  DJK Frankenberg                UBC Münster   
6    19.0   4.0  02.11.2024 20:00  DJK Frankenberg                 Hürther BC   
10   31.0   6.0  15.11.2024 20:30  DJK Frankenberg        DTV Basketball Köln   
14   43.0   8.0  07.12.2024 20:00  DJK Frankenberg     Talents BonnRhöndorf 2   
20   61.0  11.0  18.01.2025 18:00  DJK Frankenberg             Gütersloher TV   
22   71.0  12.0  25.01.2025 18:00  DJK Frankenberg         WINGS Leverkusen 2   
26   81.0  14.0  08.02.2025 18:00  DJK Frankenberg         SG Bergische Löwen   
30   96.0  16.0  22.02.2025 18:00  DJK Frankenberg    VfL AstroStars Bochum 2   
34  106.0  18.0  15.03.2025 20:00  DJK Frankenberg    New Basket Oberhausen 2   
38  116.0  20.0  29.03.2025 18:00  DJK Frankenberg                  TSV Hagen   
40  121.0  21.0  05.04.2025 18:00  DJK Frankenberg  Citybasket Recklinghausen   

                     Spielh

In [ ]:
##### formulate and solve optimization problem #####
import pulp

teamAssignedToGame = {}

problem = pulp.LpProblem('kampfgerichtsplanung', pulp.LpMinimize)

# helping lists and dictionaries
teams = schedules.keys()
games = []
for team in teams:
    for matchday in home_schedules[team]['Tag'].to_list():
        games.append((team, matchday))

# define variables
for game in games:
    for team in teams:
        teamAssignedToGame[(game, team)] = pulp.LpVariable(team+'_assigned_to_game_'+str(int(game[1]))+'_of_'+game[0], cat='Binary')
    
# objective function
problem += pulp.lpSum([teamAssignedToGame[(game, team)] for team in teams for game in games])

# constraint: exactly one team is assigned to one game
for game in games:
    problem += pulp.lpSum([teamAssignedToGame[(game, team)] for team in teams]) == 1

problem.solve()

if pulp.LpStatus[problem.status] == 'Optimal':
    for game in games:
        for team in teams:
            if teamAssignedToGame[(game, team)].varValue == True:
                print(f'Team {team} is assigned to game {game}')


damen3_assigned_to_game_2_of_damen1
herren2_assigned_to_game_4_of_damen1
damen3_assigned_to_game_6_of_damen1
herren2_assigned_to_game_8_of_damen1
herren1_assigned_to_game_11_of_damen1
herren1_assigned_to_game_12_of_damen1
herren2_assigned_to_game_14_of_damen1
damen3_assigned_to_game_16_of_damen1
damen3_assigned_to_game_18_of_damen1
herren2_assigned_to_game_20_of_damen1
herren2_assigned_to_game_21_of_damen1
herren1_assigned_to_game_2_of_damen2
herren2_assigned_to_game_4_of_damen2
damen3_assigned_to_game_6_of_damen2
u16w_assigned_to_game_8_of_damen2
damen3_assigned_to_game_12_of_damen2
herren2_assigned_to_game_14_of_damen2
herren2_assigned_to_game_16_of_damen2
herren2_assigned_to_game_18_of_damen2
damen2_assigned_to_game_20_of_damen2
damen1_assigned_to_game_9_of_damen3
herren2_assigned_to_game_2_of_damen3
damen1_assigned_to_game_4_of_damen3
u16w_assigned_to_game_6_of_damen3
damen2_assigned_to_game_10_of_damen3
u16w_assigned_to_game_16_of_damen3
herren1_assigned_to_game_17_of_damen3
damen